In [3]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
from matplotlib.pyplot import imread
import codecs
from IPython.display import HTML

movies = pd.read_csv("./data/tmdb_5000_movies.csv")
credits = pd.read_csv("./data/tmdb_5000_credits.csv")


In [4]:
import json

movies['genres'] = movies['genres'].apply(json.loads)
for index, genres in zip(movies.index, movies['genres']):
    list_genres = []
    for genre in genres:
        list_genres.append(genre['name'])
    movies.loc[index, 'genres'] = str(list_genres)

movies['keywords'] = movies['keywords'].apply(json.loads)
for index, keywords in zip(movies.index, movies['keywords']):
    list_ = []
    for keyword in keywords:
        list_.append(keyword['name'])
    movies.loc[index, 'keywords'] = str(list_)
    
# changing the production_companies column from json to string
movies['production_companies'] = movies['production_companies'].apply(json.loads)
for index,i in zip(movies.index,movies['production_companies']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))
    movies.loc[index,'production_companies'] = str(list1)

# changing the cast column from json to string
credits['cast'] = credits['cast'].apply(json.loads)
for index,i in zip(credits.index,credits['cast']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))
    credits.loc[index,'cast'] = str(list1)

# changing the crew column from json to string    
credits['crew'] = credits['crew'].apply(json.loads)
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
credits['crew'] = credits['crew'].apply(director)
credits.rename(columns={'crew':'director'},inplace=True)

In [5]:
movies = movies.merge(credits, left_on="id", right_on="movie_id", how='left')
movies.to_csv("movies_merge.csv", index=True)

movies = movies[['id','original_title','genres','cast','vote_average','director','keywords']]
movies.head()

,id,original_title,genres,cast,vote_average,director,keywords
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",7.2,James Cameron,"['culture clash', 'future', 'space war', 'spac..."
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",6.9,Gore Verbinski,"['ocean', 'drug abuse', 'exotic island', 'east..."
2,206647,Spectre,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",6.3,Sam Mendes,"['spy', 'based on novel', 'secret agent', 'seq..."
3,49026,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']","['Christian Bale', 'Michael Caine', 'Gary Oldm...",7.6,Christopher Nolan,"['dc comics', 'crime fighter', 'terrorist', 's..."
4,49529,John Carter,"['Action', 'Adventure', 'Science Fiction']","['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...",6.1,Andrew Stanton,"['based on novel', 'mars', 'medallion', 'space..."


In [97]:
movies['genres'] = movies['genres'].str.strip('[]').str.replace(' ', '').str.replace("'", '')
movies['genres'] = movies['genres'].str.split(',')

print(movies)


          id                            original_title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4798    9367                               El Mariachi   
4799   72766                                 Newlyweds   
4800  231617                 Signed, Sealed, Delivered   
4801  126186                          Shanghai Calling   
4802   25975                         My Date with Drew   

                                            genres  \
0     [Action, Adventure, Fantasy, ScienceFiction]   
1                     [Adventure, Fantasy, Action]   
2                       [Action, Adventure, Crime]   
3                 [Action, Crime, Drama, Thriller]   
4              [Action, Adventure

### Genre Column

In [98]:
genreList = []
for index, row in movies.iterrows():
    genres = row['genres']
    for genre in genres:
        if genre not in genreList:
            genreList.append(genre)

print(genreList)

['Action', 'Adventure', 'Fantasy', 'ScienceFiction', 'Crime', 'Drama', 'Thriller', 'Animation', 'Family', 'Western', 'Comedy', 'Romance', 'Horror', 'Mystery', 'History', 'War', 'Music', 'Documentary', 'Foreign', 'TVMovie', '']


In [99]:
def one_hot_encoding(x):
    encoding_list = []
    for i in range(len(genreList)):
        encoding_list.append(0)
    for i, genre in enumerate(genreList):
        if genre in x:
            encoding_list[i] = 1

    return encoding_list

movies['genres_bi'] = movies['genres'].apply(one_hot_encoding)
movies['genres_bi'].head()

0    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: genres_bi, dtype: object

### Cast column

In [100]:
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['cast'] = movies['cast'].str.split(',')

for i,j in zip(movies['cast'],movies.index):
    list2 = []
    list2 = i[:4]
    movies.loc[j,'cast'] = str(list2)
    
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['cast'] = movies['cast'].str.split(',')
for i,j in zip(movies['cast'],movies.index):
    list2 = []
    list2 = i
    list2.sort()
    movies.loc[j,'cast'] = str(list2)
movies['cast']=movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['cast'] = movies['cast'].str.split(',')

castList = []
for index, row in movies.iterrows():
    cast = row["cast"]
    
    for i in cast:
        if i not in castList:
            castList.append(i)

def binary(cast_list):
    binaryList = []
    
    for genre in castList:
        if genre in cast_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

movies['cast_bi'] = movies['cast'].apply(lambda x: binary(x))
movies['cast_bi'].head()


0    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: cast_bi, dtype: object

### Directors Column

In [101]:
print(movies['director'])

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4          Andrew Stanton
              ...        
4798     Robert Rodriguez
4799         Edward Burns
4800          Scott Smith
4801          Daniel Hsia
4802     Brian Herzlinger
Name: director, Length: 4803, dtype: object


In [102]:

director_list = []
for i, row in movies.iterrows():
    director = row['director']
    if director not in director_list:
        director_list.append(director)

def director_encoding(x):
    director_bi = []
    for director in director_list:
        if director in [x]:
            director_bi.append(1)
        else:
            director_bi.append(0)

    return director_bi

movies['director_bi'] = movies['director'].apply(director_encoding)
movies['director_bi'].head()

0    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: director_bi, dtype: object

### Keywords column

In [103]:
print(movies['keywords'][0])

['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d']


In [104]:
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'", "")
movies['keywords'] = movies['keywords'].str.split(",")

keywords_list = []
for i, row in movies.iterrows():
    keywords = row['keywords']
    for keyword in keywords:
        if keyword not in keywords_list:
            keywords_list.append(keyword)

def keyword_encoding(x):
    keyword_bi = []
    for keyword in keywords_list:
        if keyword in x:
            keyword_bi.append(1)
        else:
            keyword_bi.append(0)
    return keyword_bi

movies['keywords_bi'] = movies['keywords'].apply(keyword_encoding)
movies = movies[(movies['vote_average']!=0)] #removing the movies with 0 score and without drector names 
movies = movies[movies['director']!='']

movies['keywords_bi'].head()

0    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
Name: keywords_bi, dtype: object

#### Similarity

In [108]:
from scipy import spatial

def compute_similarity(id1, id2):
    movie_1, movie_2 = movies.iloc[id1], movies.iloc[id2]

    genre_1, genre_2 = movie_1['genres_bi'], movie_2['genres_bi']
    print(type(genre_1), genre_1)
    cast_1, cast_2 = movie_1['cast_bi'], movie_2['cast_bi']
    director_1, director_2 = movie_1['director_bi'], movie_2['director_bi']
    keywords_1, keywords_2 = movie_1['keywords_bi'], movie_2['keywords_bi']

    genre_distance = spatial.distance.cosine(genre_1, genre_2)
    cast_distance = spatial.distance.cosine(cast_1, cast_2)
    director_distance = spatial.distance.cosine(director_1, director_2)
    keywords_distance = spatial.distance.cosine(keywords_1, keywords_2)

    return genre_distance + cast_distance + 0.5*director_distance + keywords_distance

compute_similarity(3, 160)
print(movies.iloc[3])
print(movies.iloc[160])

<class 'list'> [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
original_title                                The Dark Knight Rises
genres                             [Action, Crime, Drama, Thriller]
director                                          Christopher Nolan
cast              [AnneHathaway, ChristianBale, GaryOldman, Mich...
keywords          [dccomics, crimefighter, terrorist, secretiden...
genres_bi         [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
director_bi       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
cast_bi           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...
new_id                                                            3
keywords_bi       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 3, dtype: object
original_title                           How to Train Your Dragon 2
genres            [Fantasy, Action, Adventure, Animation, Comedy...
director                                               Dean DeBlois
cast          

### Predict Score

In [106]:
new_id = list(range(0,movies.shape[0]))
movies['new_id']=new_id
movies=movies[['original_title','genres', 'director', 'cast','keywords', 'genres_bi','director_bi', 'cast_bi','new_id','keywords_bi']]

movies.to_csv("movies.csv", index=False)
movies.head()

,original_title,genres,director,cast,keywords,genres_bi,director_bi,cast_bi,new_id,keywords_bi
0,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]",James Cameron,"[SamWorthington, SigourneyWeaver, StephenLang,...","[cultureclash, future, spacewar, spacecolony, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",Gore Verbinski,"[JohnnyDepp, KeiraKnightley, OrlandoBloom, Ste...","[ocean, drugabuse, exoticisland, eastindiatrad...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Spectre,"[Action, Adventure, Crime]",Sam Mendes,"[ChristophWaltz, DanielCraig, LéaSeydoux, Ralp...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Christopher Nolan,"[AnneHathaway, ChristianBale, GaryOldman, Mich...","[dccomics, crimefighter, terrorist, secretiden...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,John Carter,"[Action, Adventure, ScienceFiction]",Andrew Stanton,"[LynnCollins, SamanthaMorton, TaylorKitsch, Wi...","[basedonnovel, mars, medallion, spacetravel, p...","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [107]:
import operator

def knn_predict_score(target_name, k=5):
    target_film = movies[movies['original_title'] == target_name]

    distances = []
    for index, row in movies.iterrows():
        if row['new_id'] != target_film["new_id"].to_list()[0]:
            distance = compute_similarity(row['new_id'], target_film['new_id'].to_list()[0])
            distances.append((row,distance))

    results = []
    distances.sort(key=operator.itemgetter(1))
    for i in range(k):
        neighbors = distances[i][0]
        results.append(neighbors['original_title'])
    
    return results

knn_predict_score('The Godfather')

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

['The Godfather: Part II',
 'The Godfather: Part III',
 'The Outsiders',
 'The Rainmaker',
 'The Cotton Club']